Aqui, começaremos a atacar o problema de sincronizar nossos arquivos de audio com os acordes escritos. Primeiramente, tentaremos, para uma dada música, determinar os acordes por compasso. Faremos isso tomando como base os arquivos em formato xml.

In [4]:
import pandas as pd
import numpy as np
import os
import xml.etree.ElementTree as ET
song_title = 'Gillette'
tree = ET.parse('corpus/xml/{}.xml'.format(song_title))
root = tree.getroot()

In [5]:
part = root.findall('part')[0]
measures = part.findall('measure')
for measure in measures:
    chords = measure.findall('harmony')
    for chord in chords:
        try:
            note = chord.find('root/root-step').text

        except AttributeError: # se não achar a nota "root" do acorde, não tem acorde
            print('Measure: {}'.format(measure.attrib['number']), ' | ', 'No chords')
            continue
            
        if note:
            # alter é a alteração de um acorde (nem sempre existe): b ou #.
            alter = chord.find('root/root-alter')#.text
            
            # kind é o tipo do acorde (nem sempre existe): pode ser com 7a, 9a, 13a, por aí vai...
            kind = chord.find('kind')#.attrib['text']
            
            # bass é o baixo do acorde, quando diferente da tônica. Também nem sempre existe.
            bass = chord.find('bass/bass-step')#.text
            
            # bass_alter é e alteração do baixo, podendo ser b ou #. Também nem sempre existe.
            bass_alter = chord.find('bass/bass-alter')#.text
            
            attribs_dict = {'alter':alter, 
                            'kind':kind, 
                            'bass':bass, 
                            'bass_alter':bass_alter}

            for key, value in attribs_dict.items():
                if value is None:
                    attribs_dict[key] = ''
                elif key == 'kind':
                    attribs_dict[key] = value.attrib['text']
                else:
                    attribs_dict[key] = value.text
                    
            print('Measure: {}'.format(measure.attrib['number']), '|', 'Chord: ', note, 
                  attribs_dict['alter'], attribs_dict['kind'],'/',attribs_dict['bass'],attribs_dict['bass_alter'])
            

Measure: 1 | Chord:  B -1  /  
Measure: 5 | Chord:  F  7 /  
Measure: 9 | Chord:  B -1  /  
Measure: 11 | Chord:  E -1  /  
Measure: 13 | Chord:  F   /  
Measure: 13 | Chord:  E  dim /  
Measure: 13 | Chord:  C  min6 /  
Measure: 13 | Chord:  D  min7 /  
Measure: 14 | Chord:  A  min /  
Measure: 14 | Chord:  F  7 /  
Measure: 15 | Chord:  B -1 7 /  
Measure: 17 | Chord:  B -1 7 /  


Comparando com a partitura aberta no musescore parece fazer sentido, só preciso conseguir fazer com que mostre também os compassos em que não tem acorde (no caso da música que estamos usando como exemplo, os compassos 1 e 2).

Agora, vamos modificar ligeiramente o código acima para adicionar as informações em um dataframe.

In [6]:
df_teste = pd.DataFrame(columns=[song_title])
df_add = pd.DataFrame(columns=[song_title], 
                      index=[3], 
                      data='E -1 min7')

df_add_2 = pd.DataFrame(columns=[song_title], 
                      index=[3], 
                      data='D -1')

In [7]:
df_teste = df_teste.append(df_add)

In [8]:
df_teste.append(df_add_2)

,Gillette
3,E -1 min7
3,D -1


In [9]:
df_harmony = pd.DataFrame(columns=[song_title])

part = root.findall('part')[0]
measures = part.findall('measure')

for measure in measures:
    chords = measure.findall('harmony')
    for chord in chords:
        try:
            note = chord.find('root/root-step').text

        except AttributeError: # se não achar a nota "root" do acorde, não tem acorde
            print('Measure: {}'.format(measure.attrib['number']), ' | ', 'No chords')
            continue
            
        if note:
            # alter é a alteração de um acorde (nem sempre existe): b ou #.
            alter = chord.find('root/root-alter')#.text
            
            # kind é o tipo do acorde (nem sempre existe): pode ser com 7a, 9a, 13a, por aí vai...
            kind = chord.find('kind')#.attrib['text']
            
            # bass é o baixo do acorde, quando diferente da tônica. Também nem sempre existe.
            bass = chord.find('bass/bass-step')#.text
            
            # bass_alter é e alteração do baixo, podendo ser b ou #. Também nem sempre existe.
            bass_alter = chord.find('bass/bass-alter')#.text
            
            attribs_dict = {'alter':alter, 
                            'kind':kind, 
                            'bass':bass, 
                            'bass_alter':bass_alter}

            for key, value in attribs_dict.items():
                if value is None:
                    attribs_dict[key] = ''
                elif key == 'kind':
                    attribs_dict[key] = value.attrib['text']
                else:
                    attribs_dict[key] = value.text
                    
                    
            measure_number = measure.attrib['number']
            chord = note + attribs_dict['alter'] + attribs_dict['kind']
            chord_bass = attribs_dict['bass']+attribs_dict['bass_alter']
            complete_chord = chord + '/' + chord_bass
            
            if complete_chord.endswith('/'):
                complete_chord = complete_chord[:-1]
            
            df_chord = pd.DataFrame(columns=[song_title], index=[measure_number], data=complete_chord)
            df_harmony = df_harmony.append(df_chord)
            
df_harmony.index.names = ['Measure']

In [10]:
df_harmony.index.unique()

Index(['1', '5', '9', '11', '13', '14', '15', '17'], dtype='object', name='Measure')

In [11]:
df_harmony.index.value_counts()

13    4
14    2
5     1
1     1
17    1
11    1
9     1
15    1
Name: Measure, dtype: int64

In [12]:
df_harmony

,Gillette
Measure,
1,B-1
5,F7
9,B-1
11,E-1
13,F
13,Edim
13,Cmin6
13,Dmin7
14,Amin


Agora vamos procurar os momentos de mudança de acorde. Vasculhando os arquivos xml que temos em nosso corpus, pudemos ver que as informações sobre andamento (em bpm) e tipo de compasso encontram-se na seção do primeiro compasso. Segue:

#### Tipo de compasso

In [62]:
first_measure = measures[0]
cima = first_measure.find('attributes').find('time/beats').text
baixo = first_measure.find('attributes').find('time/beat-type').text

compasso = cima + '/' + baixo
compasso

'2/2'

#### Andamento

In [63]:
andamento = first_measure.find('sound').attrib['tempo']# + ' bpm'
andamento
# em bpm

'300'

#### Sync: Acorde + minutagem

In [64]:
df_harmony['Time'] = ''
df_harmony

,Gillette,Time
Measure,,
1,B-1,
5,F7,
9,B-1,
11,E-1,
13,F,
13,Edim,
13,Cmin6,
13,Dmin7,
14,Amin,


In [65]:
duracao_seminima = 60/int(andamento) # em segundos

Fazendo caso a caso pra construir a generalização:

In [66]:
# if int(baixo) == 2: # a unidade é a mínima
#     segs_compasso = 2 * int(cima) * duracao_seminima
    
# elif int(baixo) == 4: # a unidade é a seminínima
#     segs_compasso = 1 * int(cima) * duracao_seminima
    
# elif int(baixo) == 8: # a unidade é a colcheia
#     segs_compasso = (1/2) * int(cima) * duracao_seminima
    
# elif int(baixo) == 16: # a unidade é a semicolcheia
#     segs_compasso = (1/4) * int(cima) * duracao_seminima

Checar se a generalização abaixo tá certa:

In [67]:
segs_compasso = (4/int(baixo)) * int(cima) * duracao_seminima

In [37]:
df_harmony.index = df_harmony.index.astype(int)

In [69]:
for compasso in df_harmony.index.unique():
    acordes = df_harmony.loc[[compasso]]
    n_acordes = len(acordes)
    print(compasso, n_acordes)
    inicio_compasso = (int(compasso) - 1) * segs_compasso
    
    if n_acordes == 1:
        tempos = inicio_compasso
    
    else:        
        tempos = np.linspace(inicio_compasso,
                             inicio_compasso + segs_compasso,
                             n_acordes+1)
        tempos = tempos[:-1]
#     df_harmony.loc[compasso]['Time'] = tempos
    
    df_harmony.at[compasso, 'Time'] = tempos

1 1
5 1
9 1
11 1
13 4
14 2
15 1
17 1


In [70]:
df_harmony

,Gillette,Time
Measure,,
1,B-1,0
5,F7,3.2
9,B-1,6.4
11,E-1,8
13,F,9.6
13,Edim,9.8
13,Cmin6,10
13,Dmin7,10.2
14,Amin,10.4


Botei pra tocar e as mudanças de acorde parecem estar fazendo sentido! 
Uma coisa que acabei de pensar é sobre os ritornellos... acho que desta maneira não os reconhecemos. Verificar depois.

## Lidando com ritornellos

In [67]:
song_title

'Gillette'

In [68]:
part = root.find('part')
measures = part.findall('measure')
for measure in measures:
    barras = measure.findall('barline')
    for barra in barras:
        
        ritornellos = barra.findall('repeat')
        for rito in ritornellos:
            print('Ritornello', measure.attrib['number'], rito.attrib['direction'])
#             print(rito)
            
        casas = barra.findall('ending')
        for casa in casas:
            print('Compasso: ',
                    measure.attrib['number'], '|',
                    'Casa nº', casa.attrib['number'], '({})'.format(casa.attrib['type']))

            


Ritornello 1 forward
Compasso:  15 | Casa nº 1 (start)
Ritornello 16 backward
Compasso:  16 | Casa nº 1 (stop)
Compasso:  17 | Casa nº 2 (start)
Compasso:  17 | Casa nº 2 (discontinue)


#### Casas
O que descobri: elas vêm como um "ending" dentro de um "barline". O interessante é que, para construir uma casa, precisamos de duas estruturas desse tipo: uma pra começar a casa e outra pra terminar. Então, por exemplo, se uma casa começa e termina num mesmo compasso, teremos dois "barlines" com "ending" dentro; um com "type" igual a "start" e outro com "type" igual a "stop" ou "descontinue" (o que vi até agora).

In [69]:
part = root.find('part')
measures = part.findall('measure')

# lista que conterá os compassos que têm ritornellos backward
ritos_back = []

# lista que conterá os compassos que têm ritornellos forward
ritos_forw = []
for measure in measures:
    barras = measure.findall('barline')
    for barra in barras:
        
        ritornellos = barra.findall('repeat')
        
        for rito in ritornellos:
            if rito.attrib['direction'] == 'backward':
                ritos_back.append(measure.attrib['number'])
            elif rito.attrib['direction'] == 'forward':
                ritos_forw.append(measure.attrib['number'])
#             print('Ritornello', measure.attrib['number'], rito.attrib['direction'])
#             print(rito)

            
        casas = barra.findall('ending')
        for casa in casas:
            print('Compasso: ',
                    measure.attrib['number'], '|',
                    'Casa nº', casa.attrib['number'], '({})'.format(casa.attrib['type']))

            


Compasso:  15 | Casa nº 1 (start)
Compasso:  16 | Casa nº 1 (stop)
Compasso:  17 | Casa nº 2 (start)
Compasso:  17 | Casa nº 2 (discontinue)


In [70]:
ritos_back

['16']

In [71]:
ritos_forw

['1']

### Numeração das casas
Aqui vamos checar como os números das casas aparecem em nosso corpus. (ex.: pode acontecer de um mesmo compasso representar as casas 1, 2 e 3, como observamos na música It's on.)

In [72]:
songs = os.listdir('corpus/xml')
ending_numbers = []
for song in songs:
#     print(song)
    tree = ET.parse('corpus/xml/{}'.format(song))
    root = tree.getroot()
    part = root.find('part')
    measures = part.findall('measure')
    for measure in measures:
        barras = measure.findall('barline')
        for barra in barras:
            casas = barra.findall('ending')
            for casa in casas:
                ending_numbers.append(casa.attrib['number'])
                if casa.attrib['number'] == '':
                    # printando casas com numeração vazia pra investigar
                    print(song)
#                 print(song)


Cold Duck B with bass.xml
Cold Duck B with bass.xml
Humanism.xml
Humanism.xml
Humanism.xml
Humanism.xml
Humanism.xml
lhcd.xml
lhcd.xml
Dangerous Curves B.xml
Dangerous Curves B.xml
Dangerous Curves B.xml
PCH.xml
Cold duck Lead.xml
Cold duck Lead.xml
ReyJorge.xml
ReyJorge.xml
ReyJorge.xml
ReyJorge.xml
HustlerTheII.xml
HustlerTheII.xml
OneShiningSoul.xml
OneShiningSoul.xml
Dangerous Curves A.xml
Dangerous Curves A.xml
Dangerous Curves A.xml


In [73]:
from collections import Counter

In [74]:
Counter(ending_numbers)

Counter({'1': 444,
         '2': 433,
         '1, 2, 3': 8,
         '4': 13,
         '': 26,
         '1, 2, 3, 4': 6,
         '3': 21,
         '5': 2,
         '1, 2, 3, 4, 5': 6,
         '1, 2': 4,
         '7': 4,
         '1, 3, 5': 2,
         '2, 4, 6': 2,
         '6': 2})

Vemos acima que é utilizado com alguma frequência o recurso de atribuir a uma casa vários números diferentes, i.e., quando uma mesma casa aparece como casas 1 e 2, por exemplo. Isso adiciona alguns desafios à nossa sincronização.

Outro lance interessante que apareceu foram casas com numeração vazia. Preciso investigar isso melhor...

Já entendi tudoooo! Casa com número vazio é quando não tem casa. Rs. Apenas o ritornello. Repete exatamente igual o que passou. Show.

#### Ideia
Precisaremos de uma estrutura que conterá infos específicas sobre as casas. Pensei em algo como uma lista cujos elementos serão dicionários, cada um relativo a uma casa. 
Para cada casa guardaremos: 
* compasso inicial
* compasso final
* compasso para onde volta
* se é a útima casa (variável booleana)

A partir disso, o formato do código será o seguinte:
Se estivermos na última casa, não faz nada (i.e.: realiza a sincronização como já havíamos estruturado anteriormente).
Se não, após o compasso final, repetimos tudo desde o compasso para onde volta até o compasso inicial - 1.

Primeiramente, vamos criar uma lista com as casas da música. O primeiro passo do código será saber qual é a última casa.

In [75]:
tree = ET.parse('corpus/xml/{}.xml'.format(song_title))
ending_numbers = []
root = tree.getroot()
part = root.find('part')
measures = part.findall('measure')

for measure in measures:
    barras = measure.findall('barline')
    for barra in barras:
        casas = barra.findall('ending')
        for casa in casas:
            n_casa = int(casa.attrib['number'])
            ending_numbers.append(n_casa)
            
ending_numbers = list(set(ending_numbers))

In [76]:
ending_numbers

[1, 2]

In [77]:
part = root.find('part')
measures = part.findall('measure')

# lista que conterá os compassos que têm ritornellos backward
ritos_back = []

# lista que conterá os compassos que têm ritornellos forward
ritos_forw = []

# lista que conterá infos sobre cada barline do tipo ending
barlines = []

for measure in measures:
    barras = measure.findall('barline')
    for barra in barras:
        
        ritornellos = barra.findall('repeat')
        
        for rito in ritornellos:
            if rito.attrib['direction'] == 'backward':
                ritos_back.append(int(measure.attrib['number']))
            elif rito.attrib['direction'] == 'forward':
                ritos_forw.append(int(measure.attrib['number']))

            
        casas = barra.findall('ending')
        for n_casa in ending_numbers:
            for casa in casas:
                if int(casa.attrib['number']) == n_casa:
                    # criando o atributo "compasso" para as casas
                    casa.attrib['measure'] = measure.attrib['number']
                    barlines.append(casa)
                    
# criando o atributo "é última casa" para as casas                    
for casa in barlines:
    
    if int(casa.attrib['number']) != len(ending_numbers):
        casa.attrib['is_last_ending'] = 0
    else:
        casa.attrib['is_last_ending'] = 1

    

Criando o dicionário que conterá as infos que precisamos de cada casa (ainda sem uma delas, que é pra qual compasso cada casa volta. Já vamos trabalhar nisso.):

In [78]:
# for barline in barlines:
#     print(barline.attrib['number'], 
#           barline.attrib['type'], 
#           barline.attrib['measure'], 
#           barline.attrib['is_last_ending'])

casas = []
for n_casa in ending_numbers:
    for barline in barlines:
        number = int(barline.attrib['number'])
        tipo = barline.attrib['type']
        
        if number == n_casa:
            if tipo == 'start':
                initial_measure = barline.attrib['measure']
                
            elif tipo == 'stop' or tipo == 'discontinue':
                final_measure = barline.attrib['measure']
         
            is_last_ending = barline.attrib['is_last_ending']
        
    dict_casa = {'number':n_casa, 
                 'initial_measure':initial_measure, 
                 'final_measure':final_measure, 
                 'is_last_ending':is_last_ending}
    
    casas.append(dict_casa)

1 start 15 0
1 stop 16 0
2 start 17 1
2 discontinue 17 1


In [79]:
casas

[{'number': 1,
  'initial_measure': '15',
  'final_measure': '16',
  'is_last_ending': 0},
 {'number': 2,
  'initial_measure': '17',
  'final_measure': '17',
  'is_last_ending': 1}]

In [80]:
ritos_back

[16]

In [81]:
ritos_forw

[1]

Checar depois se essas listas dos ritos são construídas mantendo a ordem dos números

Agora eu vou ter que pegar os compassos pra onde as casas voltam. Para isso vou ter que explorar minhas listas de ritos_back e ritos_forw.

In [82]:
for casa in casas:
    if casa['is_last_ending'] == 0:
        final_measure = int(casa['final_measure'])
        i = ritos_back.index(final_measure)
        return_measure = ritos_forw[i]
    else:
        return_measure = None
    casa['return_measure'] = return_measure

In [83]:
casas

[{'number': 1,
  'initial_measure': '15',
  'final_measure': '16',
  'is_last_ending': 0,
  'return_measure': 1},
 {'number': 2,
  'initial_measure': '17',
  'final_measure': '17',
  'is_last_ending': 1,
  'return_measure': None}]

### Refazendo o código de sincronização

Agora que nos entendemos com casas e ritornellos, vamos retomar nosso código de sincronização de harmonia e minutagem, afim de incorporar estas novas informações.

Ideia: iterar sobre as casas e pegar seu compasso final. É a partir dele que se repetirá algum pedaço da música.

In [295]:
df_harmony = pd.DataFrame(columns=[song_title,'is_first_chord_in_measure'])

part = root.findall('part')[0]
measures = part.findall('measure')

for measure in measures:
    chords = measure.findall('harmony')
    for chord in chords:
        
        i_chord = chords.index(chord)
        if i_chord == 0:
            is_first_chord = 1
        else:
            is_first_chord = 0
        
        
        try:
            note = chord.find('root/root-step').text

        except AttributeError: # se não achar a nota "root" do acorde, não tem acorde
            print('Measure: {}'.format(measure.attrib['number']), ' | ', 'No chords')
            continue
        
        if note:
            # alter é a alteração de um acorde (nem sempre existe): b ou #.
            alter = chord.find('root/root-alter')#.text
            
            # kind é o tipo do acorde (nem sempre existe): pode ser com 7a, 9a, 13a, por aí vai...
            kind = chord.find('kind')#.attrib['text']
            
            # bass é o baixo do acorde, quando diferente da tônica. Também nem sempre existe.
            bass = chord.find('bass/bass-step')#.text
            
            # bass_alter é e alteração do baixo, podendo ser b ou #. Também nem sempre existe.
            bass_alter = chord.find('bass/bass-alter')#.text
            
            attribs_dict = {'alter':alter, 
                            'kind':kind, 
                            'bass':bass, 
                            'bass_alter':bass_alter}

            for key, value in attribs_dict.items():
                if value is None:
                    attribs_dict[key] = ''
                elif key == 'kind':
                    attribs_dict[key] = value.attrib['text']
                else:
                    attribs_dict[key] = value.text
                    
                    
            measure_number = measure.attrib['number']
            chord = note + attribs_dict['alter'] + attribs_dict['kind']
            chord_bass = attribs_dict['bass']+attribs_dict['bass_alter']
            complete_chord = chord + '/' + chord_bass
            
            if complete_chord.endswith('/'):
                complete_chord = complete_chord[:-1]
                
            
            df_chord = pd.DataFrame(columns=[song_title,'is_first_chord_in_measure'], 
                                    index=[measure_number]#, 
#                                     data=[complete_chord, is_first_chord]
                                   )
            df_chord.loc[measure_number][song_title] = complete_chord
            df_chord.loc[measure_number]['is_first_chord_in_measure'] = is_first_chord
            
            df_harmony = df_harmony.append(df_chord)
            
df_harmony.index.names = ['Measure']
df_harmony.index = df_harmony.index.astype(int)

In [296]:
# resetando o index pq preciso da ordem explícita dos acordes em cada compasso, que aparecerá no novo index
df_harmony.reset_index(inplace=True)
df_harmony['Measure'] = df_harmony['Measure'].astype(int)



# aqui começa uma carteação pra fazer todos os compassos aparecerem kkkk
n_compassos = len(measures)
df_aux = pd.DataFrame(columns=['Measure',
                               'aux'])

df_aux['Measure'] = list(range(1, n_compassos+1))
# df_aux['is_first_chord_in_measure'] = ''

for i in df_aux.index:
    if df_aux.loc[i]['Measure'] not in list(df_harmony['Measure']):
#         df_aux.loc[i]['is_first_chord_in_measure'] = 1
        df_aux.at[i, 'is_first_chord_in_measure'] = 1
    else:
        pass

new_df_harmony = pd.merge(df_harmony, 
                            df_aux, 
                            on='Measure', 
                            how='outer').reset_index().sort_values(['Measure','index'])

new_df_harmony['is_first_chord_in_measure_x'] = new_df_harmony['is_first_chord_in_measure_x'].fillna(0)
new_df_harmony['is_first_chord_in_measure_y'] = new_df_harmony['is_first_chord_in_measure_y'].fillna(0)

col_1 = new_df_harmony['is_first_chord_in_measure_x']
col_2 = new_df_harmony['is_first_chord_in_measure_y']

new_df_harmony['is_first_chord_in_measure'] = col_1 + col_2

df_harmony = new_df_harmony.reset_index()[['Measure', 
                                         song_title, 
                                         'is_first_chord_in_measure']]

# df_harmony = df_harmony.reset_index()[['Measure', 
#                                        song_title, 
#                                        'is_first_chord_in_measure']]
df_harmony
# o index ta todo louco mas o que importa é que ele serviu pra ordenar os acordes dentro de um mesmo compasso
# e, para isso, ele ainda está servindo

,Measure,Gillette,is_first_chord_in_measure
0,1,B-1,1.0
1,2,NaN,1.0
2,3,NaN,1.0
3,4,NaN,1.0
4,5,F7,1.0
5,6,NaN,1.0
6,7,NaN,1.0
7,8,NaN,1.0
8,9,B-1,1.0
9,10,NaN,1.0


Problema: preciso que todos os compassos apareçam no dataframe, mesmo que não tenham acordes escritos.

Edit: Já resolvi rsrrsrsrsrs

In [297]:
casas

[{'number': 1,
  'initial_measure': '15',
  'final_measure': '16',
  'is_last_ending': 0,
  'return_measure': 1},
 {'number': 2,
  'initial_measure': '17',
  'final_measure': '17',
  'is_last_ending': 1,
  'return_measure': None}]

In [298]:
# df_harmony[df_harmony.Measure == 16].index[-1]
df_harmony[:19]

,Measure,Gillette,is_first_chord_in_measure
0,1,B-1,1.0
1,2,NaN,1.0
2,3,NaN,1.0
3,4,NaN,1.0
4,5,F7,1.0
5,6,NaN,1.0
6,7,NaN,1.0
7,8,NaN,1.0
8,9,B-1,1.0
9,10,NaN,1.0


In [299]:
# criando uma coluna nova para nos ajudar na sincronização
# ela nos dirá se o acorde vem de uma repetição e qual casa que foi responsável por ela

# df_harmony['is_repetition'] = ''
for casa in casas:
    
    if casa['is_last_ending'] == 1:
        pass
    
    else:
        comp_final = int(casa['final_measure'])
        comp_inicial = int(casa['initial_measure'])
        comp_volta = int(casa['return_measure'])

        # indice de onde dividiremos o df_harmony (meio que precisamos colocar um df no meio dele). 
        # assim dividiremos e depois juntamos de novo, com a repetição entre as 2 partes
        
        # tem um +1 no index_break pq se colocamos, por ex, df[:15], o 15 não é incluído. queremos ele dentro.
        index_break = df_harmony[df_harmony.Measure == comp_final].index[-1] + 1
        original_df_1 = df_harmony[:index_break]
        original_df_2 = df_harmony[index_break:]
        
        original_df_1['is_repetition'] = 'no'
        original_df_2['is_repetition'] = 'no'
    

        # indice do df_harmony onde o compasso pra onde volta se inicia
        index_comp_volta = df_harmony[df_harmony.Measure == comp_volta].index[0]
        index_comp_inicial = df_harmony[df_harmony.Measure == comp_inicial].index[0]
        new_df = df_harmony[index_comp_volta:index_comp_inicial]
        
        new_df['is_repetition'] = 'repetition_{}'.format(casa['number'])

        # juntando de novo
        df_harmony = original_df_1.append(new_df).append(original_df_2).reset_index()[['Measure', 
                                                                                       song_title, 
                                                                                       'is_repetition', 
                                                                                       'is_first_chord_in_measure']]

<ipython-input-299-6c05b457f65f>:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  original_df_1['is_repetition'] = 'no'
<ipython-input-299-6c05b457f65f>:24: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  original_df_2['is_repetition'] = 'no'
<ipython-input-299-6c05b457f65f>:32: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/u

In [300]:
df_harmony

,Measure,Gillette,is_repetition,is_first_chord_in_measure
0,1,B-1,no,1.0
1,2,NaN,no,1.0
2,3,NaN,no,1.0
3,4,NaN,no,1.0
4,5,F7,no,1.0
5,6,NaN,no,1.0
6,7,NaN,no,1.0
7,8,NaN,no,1.0
8,9,B-1,no,1.0
9,10,NaN,no,1.0


In [307]:
for i in df_harmony.index:
    compasso = df_harmony.loc[i]['Measure']
    repetition = df_harmony.loc[i]['is_repetition']
    
    # sacou isso aqui? se eu restringisse apenas para ser o mesmo compasso, ia dar bug com os compassos q repetem
    # por isso os acordes devem vir da mesma repetição
    df_measure = df_harmony[(df_harmony['Measure'] == compasso) & (df_harmony['is_repetition'] == repetition)]
    
    df_harmony.loc[i, 'number_of_chords_in_measure'] = len(df_measure)

In [308]:
df_harmony

,Measure,Gillette,is_repetition,is_first_chord_in_measure,number_of_chords_in_measure
0,1,B-1,no,1.0,1.0
1,2,NaN,no,1.0,1.0
2,3,NaN,no,1.0,1.0
3,4,NaN,no,1.0,1.0
4,5,F7,no,1.0,1.0
5,6,NaN,no,1.0,1.0
6,7,NaN,no,1.0,1.0
7,8,NaN,no,1.0,1.0
8,9,B-1,no,1.0,1.0
9,10,NaN,no,1.0,1.0


Note: teremos problemas com o código acima para casas que têm mais de um número. Reestruturá-lo depois. 

Uma alternativa é criar condicionais para os casos que existem no nosso corpus, dado que são poucos. Avaliar.

## 08/06: parei aqui!

Agora, após todo o processamento de ritornellos/repeticões, temos a sequência correta de acordes. Podemos, então, passar a trabalhar com a minutagem do áudio.

Tipo de compasso

In [157]:
first_measure = measures[0]
cima = first_measure.find('attributes').find('time/beats').text
baixo = first_measure.find('attributes').find('time/beat-type').text

compasso = cima + '/' + baixo
compasso

'2/2'

Andamento

In [158]:
andamento = first_measure.find('sound').attrib['tempo']# + ' bpm'
andamento
# em bpm

'300'

#### Sync

In [ ]:
df_harmony['Time'] = ''

duracao_seminima = 60/int(andamento) # em segundos
segs_compasso = (4/int(baixo)) * int(cima) * duracao_seminima

In [ ]:
for i in df_harmony.index:
    

In [ ]:
df_harmony['Time'] = ''

duracao_seminima = 60/int(andamento) # em segundos
segs_compasso = (4/int(baixo)) * int(cima) * duracao_seminima
df_harmony.index = df_harmony.index.astype(int)

for compasso in df_harmony.index.unique():
    acordes = df_harmony.loc[[compasso]]
    n_acordes = len(acordes)
    print(compasso, n_acordes)
    inicio_compasso = (int(compasso) - 1) * segs_compasso
    
    if n_acordes == 1:
        tempos = inicio_compasso
    
    else:        
        tempos = np.linspace(inicio_compasso,
                             inicio_compasso + segs_compasso,
                             n_acordes+1)
        tempos = tempos[:-1]
#     df_harmony.loc[compasso]['Time'] = tempos
    
    df_harmony.at[compasso, 'Time'] = tempos

In [60]:
df_harmony.loc[13].iloc[[-1]]

,Gillette
Measure,
13,Dmin7
